# Tiled, Parallel Image Segmentation 

Image segmentation at large scales can be both time and memory intensive. The function 'tiledSegParallel.py' (https://github.com/GeoscienceAustralia/dea-notebooks/blob/chad/segmentation/src/tiledSegParallel.py) builds upon the image segmentation algorithm developed by Shepherd et al. (2019) (implemented in the package RSGISlib) to run image segmentation across multiple cpus. A full description of their approach can be found in _Clewey et al. (2014) A Python-Based Open Source System for Geographic Object-Based Image Analysis (GEOBIA) Utilizing Raster Attribute Tables_. This script requires the installation of 'pathos.multiprocessing', a fork of python's Multiprocessing package that using Dill instead of Pickle for serializing.  

    pip install --user pathos

There are two major caveats to the use of this script.
1. As the script uses the Multiprocessing library, it cannot be run across multiple nodes. If running on Raijin, using queues that have a large number of cpus per node would be best.
2. The tiling approach is based on the bounding coordinates of the geotiff. If a geotiff is irregularly shaped such that a tile(s) contains _none_ of the input geotiff, then the segmentation will fail. If this occurs, check the ..._S1Tiles.shp_ file output during stage 1 of the algorithm. Overlay this file on top of your input geotiff to check if there are empty tiles. At the moment, the only solution is to change the extent of the geotiff to be more regularly shaped.  The 'validDataTileFraction' variable will handle tiles that contain a small fraction of the input geotiff, tiles containing less than the specified fraction are merged with a neighboring tile.  The image below shows an example of the tiling approach with merged tiles:

![Tiling Approach](https://github.com/GeoscienceAustralia/dea-notebooks/tree/chadtemp2/Supplementary_data/Images/tilingApproach.PNG)



This script was written by Chad Burton at Geoscience Australia, with advice from Pete Bunting, a co-author of RSGISlib.


### User Inputs

In [ ]:
#Location string of the geotiff you wish to segment
inputTiff = "data/nmdb_Summer2017_18_NDVI_max.tif"

#Location string of the .KEA file the geotiff will be converted too
InputKEA = "data/nmdb_Summer2017_18_NDVI_max.kea"

#Location string of clumps .KEA file that will be output 
ClumpsFile = "results/nmdb_Summer2017_18_NDVI_max_OutClumps.kea"

#Location string of the Clump zonal mean of input tiff 
meanImageTiff = "results/nmdb_Summer2017_18_NDVI_max_ClumpMean.tif"

#Location to a folder to store temporary files during segmentation
temp = 'tmps/'

#How many cpus will this run on?
ncpus=5

# what fraction of a tile should contain valid data? Below this threshold
# a tile will be merged with its neighbour. 
validDataTileFraction = 0.3

#enter the tile size parameters (in number of pixels)
width = 8000
height = 8000

### Run the cells below to conduct the image segmentation

In [ ]:
from osgeo import gdal
import os
from rsgislib.segmentation import meanImage
import rsgislib
from pathos.multiprocessing import ProcessingPool as Pool
import dill
import xarray as xr
import numpy as np
#import custom functions
import sys
sys.path.append('src')
import tiledSegParallel
from transform_tuple import transform_tuple
from SpatialTools import array_to_geotiff

In [ ]:
# Change the tiff to a kea file (only run this once!)
gdal.Translate(InputKEA, inputTiff, format='KEA', outputSRS='EPSG:3577')

In [ ]:
#run the segmentation
tiledSegParallel.performTiledSegmentation(InputKEA, ClumpsFile, tmpDIR=temp, numClusters=20, validDataThreshold=validDataTileFraction, 
                                          tileWidth=width, tileHeight=height, minPxls=100, ncpus=ncpus)

In [ ]:
# Attribute segments with zonal mean of input image and output as geotiff
meanImage(inputTiff, ClumpsFile, meanImageTiff, "GTIFF",rsgislib.TYPE_32FLOAT)

In [ ]:
#convert segments result into geotiff (gdal.Translate is failing for some reason?)
a = xr.open_rasterio(ClumpsFile).squeeze()
transform, projection = transform_tuple(a, (a.x, a.y), epsg=3577)
width,height = a.shape
array_to_geotiff(ClumpsFile[:-4]+".tif",
      a.values, geo_transform = transform, 
      projection = projection, 
      nodata_val=-999)